In [ ]:
# 回到 FINRL/
%cd ..

In [ ]:
from AI_Trading.src import config
from AI_Trading.src.env_portfolio_allocation import *
from AI_Trading.src.evaluate import *
from AI_Trading.src import model_config
from AI_Trading.src.preprocess import *
from AI_Trading.src.testPortfolio import *
from AI_Trading.src.train import *
from AI_Trading.src.augmentation import *
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
import pandas as pd
from tqdm import tqdm
from warnings import simplefilter
simplefilter(action='ignore')
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [ ]:
create_dir()

In [ ]:
#evaluate setting
model_name = 'DDPG'
exp = 'DDPG_OHLC(closeNormalized)&macd(all)(minmax)_riskConcern_alpha-10_3y_window20_until2022'
save_path = os.path.join(config.RESULTS_DIR, exp)
training_log_path = f'{config.LOG_PATH}/{exp}/training_log_{exp}_0.csv'
training_weight_path = f'{config.LOG_PATH}/{exp}/training_weight_{exp}_0.csv'
training_share_path = f'{config.LOG_PATH}/{exp}/training_share_{exp}_0.csv'
if not os.path.isdir(save_path):
     try:
          os.mkdir(save_path)
     except Exception:
          print(f'no folder {save_path}')
          pass

In [ ]:
# test
for i in tqdm(range(len(config.TRAIN_START_DATE))):
    train,trade = preprocess(config.TRAIN_START_DATE[0], config.TRAIN_END_DATE[i], config.TEST_START_DATE[i], config.TEST_END_DATE[i], window=config.ADD_WINDOW, cov=False)
    env_kwargs = {
        "training_log_path": training_log_path,
        "training_weight_path": training_weight_path,
        "training_share_path": training_share_path,
        "hmax": 100, 
        "initial_amount": config.INITIAL_AMOUNT, 
        "transaction_cost_pct": 0.001, 
        "state_space": len(train.tic.unique())+4, 
        "stock_dim": len(train.tic.unique()), 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": len(train.tic.unique()), 
        "reward_scaling": 1e-4,
        "add_cash": False,
        "lookback":config.LOOKBACK,
        "alpha": -10,
        "add_window": config.ADD_WINDOW,
        "cov": False,
        "reward_type": 'riskConcern'
    }
    print(env_kwargs)
    print('config.add_window:', config.ADD_WINDOW)

    # env_train = blackLittermanEnv(df = train, is_test_set=False, **env_kwargs)
    # env_trade = blackLittermanEnv(df = trade, is_test_set=True, **env_kwargs)
    env_train = windowEnv(df = train, is_test_set=False, **env_kwargs)
    env_trade = windowEnv(df = trade, is_test_set=True, **env_kwargs)
    
    model_index = i

    # load model
    model_zip_path = f'{config.TRAINED_MODEL_PATH}/{exp}/{model_name}_{str(model_index)}.zip'
    if os.path.exists(model_zip_path):
        if model_name == 'A2C':
            model = A2C.load(model_zip_path)
        elif model_name == 'PPO':
            model = PPO.load(model_zip_path)
        elif model_name == 'DDPG':
            model = DDPG.load(model_zip_path,seed=0)
        elif model_name == 'TD3':
            model = TD3.load(model_zip_path)
        elif model_name == 'SAC':
            model = SAC.load(model_zip_path)
    # test
    df_daily_return, df_actions = test_portfolioAllocation(model, env_trade)
    df_actions.to_csv(f'{save_path}/df_action_{model_name}_{model_index}.csv')
